In [70]:
import csv # not needed as pandas is being used
import numpy as np
import pandas as pd
import pylab as P # for the graphs/histograms

titanic_dataframe = pd.read_csv('train.csv', header=0) # header = 0 since initial row is a header
test = pd.read_csv('test.csv',header=0)


In [22]:
# ---------------- Playing with the dataframe --------

# print titanic_dataframe
# print titanic_dataframe.head(5)

# Checking data types
# print titanic_dataframe.dtypes # Pandas infers numerical types

# Check how many non-null entries
# print titanic_dataframe.info()

# Check means, deviations and quartiles
# print titanic_dataframe.describe()

# print titanic_dataframe[titanic_dataframe['Age']<10]

# Check rows where age is null (i.e., missing data)
# print titanic_dataframe[titanic_dataframe['Age'].isnull()]

# for i in range(1,4):
#     print i, len(titanic_dataframe[(titanic_dataframe['Sex']=='male') & (titanic_dataframe['Pclass'] == i)])

# titanic_dataframe['Pclass'].hist(bins=3, range=(0,4), alpha=0.5)
# P.show()

# titanic_dataframe['Age'].hist()
# P.show()
# ---------------- -----------------------------------

In [71]:
# Insert a new column into dataframe
# titanic_dataframe['gender'] = 10

# Use lambda fns to do the same
# Map text to binary vals

# Mapping sex to binary values (in a new column gender)
titanic_dataframe['gender'] = titanic_dataframe["Sex"].map( {"male":1, "female":0} ).astype(int)
print titanic_dataframe['gender'].head()

test['gender'] = test['Sex'].map( {'male':1, 'female':0} ).astype(int)

# Mapping embarked to float values
titanic_dataframe['boardedAt'] = titanic_dataframe['Embarked'].map( {'S': 1 , 'C':2, 'Q': 3} )
print titanic_dataframe['boardedAt'].head()
test['boardedAt'] = test['Embarked'].map( {'S':1, 'C':2, 'Q':3} ).astype(float)

print titanic_dataframe.info()
print test.info()

0    1
1    0
2    0
3    0
4    1
Name: gender, dtype: int64
0    1
1    2
2    1
3    1
4    1
Name: boardedAt, dtype: float64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 14 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
gender         891 non-null int64
boardedAt      889 non-null float64
dtypes: float64(3), int64(6), object(5)
memory usage: 104.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 417
Data columns (total 13 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Se

In [72]:
# Handle missing ages
median_ages = np.zeros((2,3))

for i in range(0,2):
    for j in range(0,3):
        median_ages[i,j]= titanic_dataframe[(titanic_dataframe['gender']==i) & \
                                            (titanic_dataframe['Pclass']==j+1)]['Age'].dropna().median()
print median_ages

# fill missing values
titanic_dataframe['filledAge'] = titanic_dataframe['Age']
test['filledAge'] = test['Age']

for i in range(0,2):
    for j in range(0,3):
        titanic_dataframe.loc[ (titanic_dataframe.Age.isnull()) \
                             & (titanic_dataframe.gender == i) \
                             & (titanic_dataframe.Pclass == j+1), \
                             'filledAge'] = median_ages[i,j]
        test.loc[ (test.Age.isnull()) & (test.gender == i) & (test.Pclass==j+1), 'filledAge' ] = median_ages[i,j]

# Check if missing ages are filled
print titanic_dataframe[ titanic_dataframe['Age'].isnull() ][ ['gender', 'Pclass', 'Age', 'filledAge'] ].head()
print test[test['Age'].isnull()][['gender','Pclass', 'Age', 'filledAge']].head()

[[ 35.   28.   21.5]
 [ 40.   30.   25. ]]
    gender  Pclass  Age  filledAge
5        1       3  NaN       25.0
17       1       2  NaN       30.0
19       0       3  NaN       21.5
26       1       3  NaN       25.0
28       0       3  NaN       21.5
    gender  Pclass  Age  filledAge
10       1       3  NaN       25.0
22       0       1  NaN       35.0
29       1       3  NaN       25.0
33       0       3  NaN       21.5
36       0       3  NaN       21.5


In [74]:
# Feature Engineering 

titanic_dataframe['famSize'] = titanic_dataframe['SibSp'] + titanic_dataframe['Parch']
test['famSize'] = test['SibSp']+test['Parch']
# print titanic_dataframe.famSize.head(20)
titanic_dataframe['famSize'].hist()
P.show()

# Create additional useful features

# Old => less likely to survive. 3rd class also implies the same
titanic_dataframe['ageXclass'] = titanic_dataframe.filledAge * \
                                    titanic_dataframe.Pclass
test['ageXclass'] = test.filledAge * test.Pclass

print titanic_dataframe[['ageXclass', 'Age', 'Pclass']].head(20)
print test[['ageXclass', 'Age', 'Pclass']].head(20)
titanic_dataframe['ageXclass'].hist()
P.show()

    ageXclass  Age  Pclass
0        66.0   22       3
1        38.0   38       1
2        78.0   26       3
3        35.0   35       1
4       105.0   35       3
5        75.0  NaN       3
6        54.0   54       1
7         6.0    2       3
8        81.0   27       3
9        28.0   14       2
10       12.0    4       3
11       58.0   58       1
12       60.0   20       3
13      117.0   39       3
14       42.0   14       3
15      110.0   55       2
16        6.0    2       3
17       60.0  NaN       2
18       93.0   31       3
19       64.5  NaN       3
    ageXclass   Age  Pclass
0       103.5  34.5       3
1       141.0  47.0       3
2       124.0  62.0       2
3        81.0  27.0       3
4        66.0  22.0       3
5        42.0  14.0       3
6        90.0  30.0       3
7        52.0  26.0       2
8        54.0  18.0       3
9        63.0  21.0       3
10       75.0   NaN       3
11       46.0  46.0       1
12       23.0  23.0       1
13      126.0  63.0       2
14       47.0

In [112]:
# print titanic_dataframe.dtypes[ titanic_dataframe.dtypes.map(lambda x:x=='object') ]
# print titanic_dataframe.info()

try:
    
    titanic_dataframe = titanic_dataframe.drop(['Name','Sex','Ticket', 'Cabin', 'Embarked', 'Age'], axis=1) 
    dataset = titanic_dataframe.values

    test = test.drop(['Name','Sex','Ticket', 'Cabin', 'Embarked', 'Age'], axis=1) 
    testdata = test.values
except:
    pass

titanic_dataframe = titanic_dataframe.dropna()

xTrain = titanic_dataframe.drop(['PassengerId','Survived'],axis=1)
xTrain = xTrain._get_numeric_data()
print xTrain.shape

yTrain = titanic_dataframe.Survived
print yTrain.shape

xTest = test.drop(['PassengerId'],axis=1)
xTest = xTest._get_numeric_data()
print xTest.shape

# print test.info()
# print titanic_dataframe.info()

# print testdata
# dataset[0::,1] # Starting at 0th row, every 1st column element (i.e., survived column)
# dataset[0::,2::] # The training data

(889, 9)
(889,)
(418, 9)


In [132]:
# Using the scikit learn package to perform the final classification
from sklearn.ensemble import RandomForestClassifier

# There's a missing value for fare in the test data set
xTest["Fare"].fillna(xTest['Fare'].median(), inplace=True)

forest1 = RandomForestClassifier(n_estimators = 200, criterion="entropy", )
forest1 = forest1.fit(xTrain[:],yTrain[:])
pred_forest1 = forest1.predict(xTest)
# print forest1.predict(xTest[150:170])
# print pred_forest1
# print len(pred_forest1)
print forest1.feature_importances_

[ 0.06239682  0.03552987  0.020758    0.24153086  0.20405867  0.03397621
  0.15266165  0.0518867   0.19720122]


In [133]:
# Making the final submission

# Create a new data frame as required for the final submission
submission = pd.DataFrame( {
        "PassengerId": test.PassengerId,
        "Survived": pred_forest1
    })

# Convert data frame to csv
submission.to_csv('titanic_preds_2.csv', index=False)
